In [2]:
import pandas as pd
import numpy as np

import matplotlib
from matplotlib import pyplot as plt

states = ('Weight1', 'Weight2', 'Weight3', 'Weight4')
observations = ('Path1', 'Path2', 'Path3', 'Path4')

#V = np.array([3, 3, 2, 1, 2, 2, 1, 2, 3, 0])
#Path1, Path2, Path3, Path4 = 1, 2, 3, 4
obs_seq = [4, 4, 3, 2, 3, 3, 2, 3, 4, 1]


#initial probability 
pi = np.array((0.2, 0.2, 0.2, 0.4))

#Transmission probability 
A = np.array(((0.3, 0.2, 0.2, 0.3), (0.2, 0.3, 0.2, 0.3), (0.2, 0.2, 0.3, 0.3), (0.2, 0.2, 0.2, 0.4)))

#Emission probability
B = np.array(((0.3, 0.3, 0.3, 0.1), (0.3, 0.3, 0.3, 0.1), (0.3, 0.3, 0.3, 0.1), (0.1, 0.1, 0.1, 0.7)))



def forward(pi, A, B, obs_seq):
        T = len(obs_seq)
        N = A.shape[0]
        alpha = np.zeros((T, N))
        alpha[0] = pi*B[:,obs_seq[0]]
        for t in range(1, T):
            alpha[t] = alpha[t-1].dot(A) * B[:, obs_seq[t]]
        return alpha

def likelihood(obs_seq):
        # returns log P(Y  \mid  model)
        # using the forward part of the forward-backward algorithm
        return  forward(obs_seq)[-1].sum()      

    
def backward(A, B, obs_seq):
        N = A.shape[0]
        T = len(obs_seq)

        beta = np.zeros((N,T))
        beta[:,-1:] = 1

        for t in reversed(range(T-1)):
            for n in range(N):
                beta[n,t] = np.sum(beta[:,t+1] * A[n,:] * B[:, obs_seq[t+1]])

        return beta

def gamma(obs_seq):
    alpha = forward(obs_seq)
    beta  = backward(obs_seq)
    obs_prob = likelihood(obs_seq)
    return (np.multiply(alpha,beta.T) / obs_prob)

def viterbi(pi, A, B, obs_seq):
        # returns the most likely state sequence given observed sequence x
        # using the Viterbi algorithm$
        global delta, psi
        T = len(obs_seq)
        N = A.shape[0]
        delta = np.zeros((T, N))
        psi = np.zeros((T, N))
        delta[0] = pi*B[:,obs_seq[0]]
        for t in range(1, T):
            for j in range(N):
                delta[t,j] = np.max(delta[t-1]*A[:,j]) * B[j, obs_seq[t]]
                psi[t,j] = np.argmax(delta[t-1]*A[:,j])

        # backtrack
        states = np.zeros(T, dtype=np.int32)
        states[T-1] = np.argmax(delta[T-1])
        for t in range(T-2, -1, -1):
            states[t] = psi[t+1, states[t+1]]
        return states

#https://sambaiga.github.io/2017/05/03/hmm-intro.html

In [3]:
viterbi(pi, A, B, observations)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [24]:
delta

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [25]:
psi

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [26]:
len(obs_seq)

10

In [27]:
forward(pi, A, B, obs_seq)

IndexError: index 4 is out of bounds for axis 1 with size 4

In [28]:
obs_seq

[4, 4, 3, 2, 3, 3, 2, 3, 4, 1]